In [3]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Define Physics-Informed Neural Network (PINN)
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.hidden_layers = nn.Sequential(
            nn.Linear(4, 50),  # 4 input parameters (x, y, z, t)
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 3)  # 3 output parameters (u, v, w)
        )

    def forward(self, X):
        return self.hidden_layers(X)

# Compute derivatives using autograd
def compute_gradients(f, wrt):
    return autograd.grad(f, wrt, grad_outputs=torch.ones_like(f), create_graph=True)[0]

# Define PDE residual function
def burgers_pde_residual(model, x, y, z, t, nu):
    # Ensure inputs have requires_grad=True
    x.requires_grad = True
    y.requires_grad = True
    z.requires_grad = True
    t.requires_grad = True

    X = torch.cat((x, y, z, t), dim=1)  # Shape (N,4)
    u_pred = model(X)  # Get predictions
    u, v, w = u_pred[:, 0:1], u_pred[:, 1:2], u_pred[:, 2:3]

    # Compute first-order derivatives
    u_x = compute_gradients(u, x)
    u_y = compute_gradients(u, y)
    u_z = compute_gradients(u, z)
    u_t = compute_gradients(u, t)

    v_x = compute_gradients(v, x)
    v_y = compute_gradients(v, y)
    v_z = compute_gradients(v, z)
    v_t = compute_gradients(v, t)

    w_x = compute_gradients(w, x)
    w_y = compute_gradients(w, y)
    w_z = compute_gradients(w, z)
    w_t = compute_gradients(w, t)

    # Compute second-order derivatives
    u_xx = compute_gradients(u_x, x)
    u_yy = compute_gradients(u_y, y)
    u_zz = compute_gradients(u_z, z)

    v_xx = compute_gradients(v_x, x)
    v_yy = compute_gradients(v_y, y)
    v_zz = compute_gradients(v_z, z)

    w_xx = compute_gradients(w_x, x)
    w_yy = compute_gradients(w_y, y)
    w_zz = compute_gradients(w_z, z)

    # Compute residuals of the PDE
    res_u = u_t + u * u_x + v * u_y + w * u_z - nu * (u_xx + u_yy + u_zz)
    res_v = v_t + u * v_x + v * v_y + w * v_z - nu * (v_xx + v_yy + v_zz)
    res_w = w_t + u * w_x + v * w_y + w * w_z - nu * (w_xx + w_yy + w_zz)

    return res_u, res_v, res_w

# Define Training Data
x = torch.linspace(-1, 1, 20)
y = torch.linspace(-1, 1, 20)
z = torch.linspace(-1, 1, 20)
t = torch.linspace(0, 1, 10)

x_train, y_train, z_train, t_train = torch.meshgrid(x, y, z, t, indexing='ij')

# Reshape and stack the input data
x_train = x_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
z_train = z_train.reshape(-1, 1)
t_train = t_train.reshape(-1, 1)

train_data = torch.cat([x_train, y_train, z_train, t_train], dim=1).float()  # Convert to float tensor

# Initialize the model, optimizer, and loss function
model = PINN()
optimizer = optim.Adam(model.parameters(), lr=0.01)
nu = 0.01  # Viscosity



In [ ]:

# Training Loop
num_epochs = 12000
print("Starting Training...")

for epoch in range(num_epochs):
    model.train()

    res_u, res_v, res_w = burgers_pde_residual(model, x_train, y_train, z_train, t_train, nu)
    loss_res = torch.mean(res_u**2 + res_v**2 + res_w**2)

    # Total loss
    loss_total = loss_res

    optimizer.zero_grad()
    loss_total.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {loss_total.item()}")

print("Training Complete!")



Starting Training...
Epoch 0, Loss: 0.0011494120117276907


In [2]:
# Plotting function for the results
import torch

import matplotlib.pyplot as plt
def plot_results(model, x, y, z, t):
    X_test = torch.cat((x, y, z, t), dim=1)
    model.eval()
    with torch.no_grad():
        u_pred = model(X_test)

    u = u_pred[:, 0].reshape(len(x), len(y), len(z), len(t)).cpu().numpy()

    plt.figure(figsize=(10, 6))
    plt.imshow(u[:, :, 10, 5], cmap='jet')
    plt.colorbar()
    plt.title('Predicted Velocity u(x, y, z, t)')
    plt.show()

# Generate test points for visualization
x_test = torch.linspace(-1, 1, 20).reshape(-1, 1)
y_test = torch.linspace(-1, 1, 20).reshape(-1, 1)
z_test = torch.linspace(-1, 1, 20).reshape(-1, 1)
t_test = torch.linspace(0, 1, 10).reshape(-1, 1)

plot_results(model, x_test, y_test, z_test, t_test)

NameError: name 'model' is not defined